# Import All Modules

In [56]:
# Enable Intellisense for code assisatance and autocomplete
%config IPCompleter.greedy=True

#Install standard modules
import pandas as pd
import numpy as np

# Time modules
import datetime as dt
import datetime
from datetime import timedelta, date, datetime
from dateutil.relativedelta import relativedelta
import time
import calendar


# Yahoo scrapper module
from yahoo_earnings_calendar import YahooEarningsCalendar

# Timezone modules
import pytz

# Progress bar modules
from tqdm.notebook import tqdm

# working days and public holidays module
import pandas_market_calendars as mcal

# API modules
import requests

# json convertor
from pandas.io.json import json_normalize

# Plotting
import matplotlib.pyplot as plt


In [57]:
# Input AMeritrade API key
api_key = open("Ameritrade_API_key.txt", "r").read()

# Function checking all potential trades with High ATR and Volatilities in Backwardation

In [70]:
def option_chain_volalitilty(ticker,
                             min_und=30,                    #Cut off limit for the minimum underlying stock price
                             max_und=1500,                  #Cut off limit for the maximum underlying stock price
                             atr_percentage_limit=4,        #Cut off limit for the relative ATR percentage
                             volatility_percentage_diff=2,  #Cut off limit for the minimum relative volatility difference between the front and back weekly options
                             min_open_int=100,              #Cut off limit for the minimum Open Interest
                             cut_off_strike_percent=3       #Cut off limit for the minimum Strike difference in percentage
                            ):

    
    # get today's date and date in 2 weeks
    todays_date = date.today() + timedelta(days=3)
    end_date = todays_date + timedelta(days=17)
    
    #define number of strikes
    strike_number = 2

     # define our endpoint
    endpoint = r"https://api.tdameritrade.com/v1/marketdata/chains"

     # define our payload
    payload = {'apikey' : api_key,
               'symbol' : ticker,
               'contractType':'ALL',
               'includeQuotes':'TRUE',
               'strategy':'SINGLE',
               'strikeCount' : strike_number,
               'fromDate':todays_date,
               'toDate':end_date,
               'optionType':'S'
               }

     # make a request
    content = requests.get(url = endpoint, params = payload)

    # convert it to dictionary
    data = content.json()
    
    # Check if payload was succesful
    if data['status'] == 'FAILED':
        pass
    else:
        if data['underlying']['mark'] < min_und and data['underlying']['mark']<max_und:
            #print(ticker,": underlying too low")
            pass
            
        else:
            calls = pd.json_normalize(data['callExpDateMap'])
            puts = pd.json_normalize(data['putExpDateMap'])

            if data["numberOfContracts"] < (4*strike_number):
                #print(ticker, ": error pulling data")
                pass

            else:
                ### Front
                # Define first Strike
                front_first_strike_call = calls.loc[0][0][0]
                front_first_strike_put = puts.loc[0][0][0]
                # Define second Strik
                front_second_strike_call = calls.loc[0][1][0]
                front_second_strike_put = puts.loc[0][1][0]
    
                ### Back
                # Define first Strike
                back_first_strike_call = calls.loc[0][2][0]
                back_first_strike_put = puts.loc[0][2][0]
                # Define second Strike
                back_second_strike_call = calls.loc[0][3][0]
                back_second_strike_put = puts.loc[0][3][0]
              
                # Calculate average volatility at the money
                avg_front_volatility = round((front_first_strike_call['volatility']+front_second_strike_call['volatility']+front_first_strike_put['volatility']+front_second_strike_put['volatility'])/4,1)
                avg_back_volatility = round((back_first_strike_call['volatility']+back_second_strike_call['volatility']+back_first_strike_put['volatility']+back_second_strike_put['volatility'])/4,1)
                percentage_difference = round(((avg_front_volatility-avg_back_volatility)/avg_back_volatility)*100,1)
                
                # Convert timestamps
                front_expirey = pd.to_datetime(front_first_strike_call['expirationDate'],unit='ms')       
                back_expirey = pd.to_datetime(back_first_strike_call['expirationDate'],unit='ms')   
                
                # Check strike regularity
                underlying = data['underlying']['mark']
                strike_diff = front_second_strike_put['strikePrice']-front_first_strike_put['strikePrice']
                strike_diff_percentage = round((strike_diff/underlying)*100,1)
                if strike_diff_percentage > cut_off_strike_percent:
                    #print(ticker,': not enough strikes')
                    pass
                else:
                    
                    # Make sure expiry dates are not the same
                    if front_expirey == back_expirey:
                        #print(ticker, ": Error - Expiry dates the same!")
                        pass
                    else:
                        # Filter out volatilities in Contango with a 5% control
                        if percentage_difference < volatility_percentage_diff:
                            #print(ticker,': Volatility in Contango')
                            pass
                        else:
                            # Calculate average open interest in the money
                            avg_open_interest_front_ITM = round((front_second_strike_call['openInterest']+front_first_strike_put['openInterest'])/2)
                            avg_open_interest_back_ITM = round((back_second_strike_call['openInterest']+back_first_strike_put['openInterest'])/2)                    

                            # Filter out open Interests
                            if avg_open_interest_front_ITM < min_open_int or avg_open_interest_back_ITM < min_open_int:
                                #print(ticker, ': Open Interest too low')
                                pass
                            else:

                                atr = historical_atr(ticker)
                                atr_percentage = round((atr/underlying)*100,1)
                                #filter out low ATR trades
                                if atr_percentage < atr_percentage_limit:
                                    #print(ticker,': ATR too low')
                                    pass
                                else:
                                    expected_move = ((front_first_strike_put['bid'] + front_second_strike_call['bid'])/underlying)*100
                                    print("----" , ticker, "----")
                                    print('Underlying: ', underlying," $")
                                    print('ATR: ', round(atr,1))
                                    print('ATR Percentage: ', atr_percentage," %")
                                    print('Expected Move: ', round(expected_move,1)," %")
                                    print('Volatility diffenece: ',percentage_difference," %")
                                    print('Strike difference percentage: ',strike_diff_percentage,"%")
                                    print("")

                                    #Front week
                                    print("  Straddle: Front Week")
                                    print("  Avg volatility: ",avg_front_volatility,"%")
                                    print("  Front exp date: ", str(front_expirey)[:10])
                                    print("  Avg Open interest ITM: ",avg_open_interest_front_ITM)
                                    print("")

                                    #Back week             
                                    print("  Straddle: Back Week")
                                    print("  Avg volatility: ", avg_back_volatility,"%")
                                    print("  Back exp date: ",str(back_expirey)[:10])
                                    print("  Avg open interest ITM: ",avg_open_interest_back_ITM)
                                    print("")

    # wait x seconds
    time.sleep(0.5)
    return 

# Historical ATR checker

In [71]:
# Calculate and return the 14 day Average True Range (ATR) of the stocks
def historical_atr(ticker):

     # define our endpoint
    endpoint = r"https://api.tdameritrade.com/v1/marketdata/{}/pricehistory".format(ticker)

     # define our payload
    payload = {'apikey':api_key,
               'periodType': 'year',
               'period' : '1',
               'frequencyType':'daily'
              }

     # make a request
    content = requests.get(url = endpoint, params = payload)

    # calculate ATR
    data = content.json()
    data = pd.json_normalize(data['candles'])
    data['high_close'] = abs(data['close'].shift(1).subtract(data['high']))
    data['low_close'] = abs(data['close'].shift(1).subtract(data['low']))
    data['high_low'] = abs(data['high'].shift(1).subtract(data['low']))
    data['tr'] = data[['high_close','low_close','high_low']].max(axis=1)
    data['atr_14d'] = data['tr'].ewm(alpha=1/14, min_periods=14,adjust=False).mean()
    data['atr_14d_percent'] = (data['atr_14d']/((data['open']+data['close'])/2))*100
    data['hist_vol_30d'] = ((np.log(data['close'].shift(1)/data['close'])).rolling(window=30).std()*np.sqrt(252)*100)
    atr = data['atr_14d'].iloc[-1]

    #data[['close' ,'hist_vol_30d','atr_14d_percent']].plot(subplots=True,figsize=(8, 6))

    time.sleep(0.5)

    return atr

# Running the pre-defined functions for trades of the week

First import all tickers that have weekly stocks

In [72]:
tickers_with_weekly = list(pd.read_csv("weekly_stocks_to_check.csv",index_col=0))

In [73]:
tickers_with_weekly

[]

In [74]:
historical_atr('MSFT')

7.557080310139808

In [75]:
temp_list = ['AMZN','GM','PINS','SNAP','XOM']

In [76]:
def option_chain_skew(ticker):

    # get today's date and date in 2 weeks
    todays_date = date.today() + timedelta(days=8)
    end_date = todays_date + timedelta(days=7)
    

     # define our endpoint
    endpoint = r"https://api.tdameritrade.com/v1/marketdata/chains"

     # define our payload
    payload = {'apikey' : api_key,
               'symbol' : ticker,
               'contractType':'ALL',
               'includeQuotes':'TRUE',
               'strategy':'SINGLE',
               'fromDate':todays_date,
               'toDate':end_date,
               'optionType':'S'
               }

     # make a request
    content = requests.get(url = endpoint, params = payload)

    # convert it to dictionary
    data = content.json()
    calls = pd.json_normalize(data['callExpDateMap'])
    puts = pd.json_normalize(data['putExpDateMap'])
    
    # Check if payload was succesful
    if data['status'] == 'FAILED':
        pass
    else:

        # 
        i=0
        while True:
            if float(puts.loc[0][i][0]['delta']) <= -0.2:
                straddle_delta_20_strike = puts.loc[0][i][0]['strikePrice']
                put_straddle_bid = puts.loc[0][i][0]['bid']
                straddle_strike = puts.loc[0][i][0]['strikePrice']
                break
            i+=1

        i=0
        while True:
            if float(calls.loc[0][i][0]['strikePrice']) == straddle_strike:
                call_straddle_bid = calls.loc[0][i][0]['bid']
                break
            i+=1

        i=0    
        while True:
            if float(puts.loc[0][i][0]['delta']) <= -0.5:
                put_straddle_bid_atm = puts.loc[0][i][0]['bid']
                straddle_strike_atm = puts.loc[0][i][0]['strikePrice']
                break
            i+=1

        i=0
        while True:
            if float(calls.loc[0][i][0]['strikePrice']) == straddle_strike_atm:
                call_straddle_bid_atm = calls.loc[0][i][0]['bid']
                break
            i+=1

        straddle_delta_20 = call_straddle_bid + put_straddle_bid
        straddle_delta_atm = put_straddle_bid_atm + call_straddle_bid_atm
        skew_percent = round(((straddle_delta_20/straddle_delta_atm)-1)*100,1)

        if skew_percent < 20:
            return False
        
        elif skew_percent < 25:
            min_credit_percent = 28
        elif skew_percent < 30:
            min_credit_percent = 27
        elif skew_percent < 35:
            min_credit_percent = 26
        elif skew_percent < 40:
            min_credit_percent = 25       
        elif skew_percent < 45:
            min_credit_percent = 24
        elif skew_percent < 50:
            min_credit_percent = 23
        elif skew_percent < 55:
            min_credit_percent = 22        
        elif skew_percent < 60:
            min_credit_percent = 21
        elif skew_percent < 65:
            min_credit_percent = 20        
        elif skew_percent < 70:
            min_credit_percent = 19
        elif skew_percent < 75:
            min_credit_percent = 18
        else:
            min_credit_percent = 17
 
        print('---',ticker,'---')
        print(' straddle_delta_20_strike: ',straddle_strike)
        print(" straddle_delta_20:        ",round(straddle_delta_20,2))
        print(' straddle_delta_atm_strike:', straddle_strike_atm)
        print(' straddle_delta_atm:       ', round(straddle_delta_atm,2))
        print(' skew_percent:             ', skew_percent,'%')
        print(' min_credit_percent:       ',min_credit_percent)
        print("")
            
            
        time.sleep(0.5)
    return min_credit_percent

In [77]:
for i in tqdm(temp_list):
    option_chain_skew(i)

--- AMZN ---
 straddle_delta_20_strike:  2980.0
 straddle_delta_20:         176.15
 straddle_delta_atm_strike: 3125.0
 straddle_delta_atm:        132.35
 skew_percent:              33.1 %
 min_credit_percent:        26

--- GM ---
 straddle_delta_20_strike:  37.5
 straddle_delta_20:         2.54
 straddle_delta_atm_strike: 40.0
 straddle_delta_atm:        2.07
 skew_percent:              22.7 %
 min_credit_percent:        28

--- PINS ---
 straddle_delta_20_strike:  55.5
 straddle_delta_20:         5.41
 straddle_delta_atm_strike: 60.0
 straddle_delta_atm:        4.36
 skew_percent:              24.1 %
 min_credit_percent:        28

--- XOM ---
 straddle_delta_20_strike:  34.0
 straddle_delta_20:         2.33
 straddle_delta_atm_strike: 36.0
 straddle_delta_atm:        1.61
 skew_percent:              44.7 %
 min_credit_percent:        24




In [78]:
def short_strangle_scanner(ticker):

    # get today's date and date in 2 weeks
    todays_date = date.today() + timedelta(days=3)
    end_date = todays_date + timedelta(days=8)
    

     # define our endpoint
    endpoint = r"https://api.tdameritrade.com/v1/marketdata/chains"

     # define our payload
    payload = {'apikey' : api_key,
               'symbol' : ticker,
               'contractType':'ALL',
               'includeQuotes':'TRUE',
               'strategy':'SINGLE',
               'fromDate':todays_date,
               'toDate':end_date,
               'optionType':'S'
               }

     # make a request
    content = requests.get(url = endpoint, params = payload)

    # convert it to dictionary
    data = content.json()
    
    # Check if payload was succesful
    if data['status'] == 'FAILED':
        pass
    else:
        calls = pd.json_normalize(data['callExpDateMap'])
        puts = pd.json_normalize(data['putExpDateMap'])

        # get straddle price atm
        i=0    
        while True:
            if float(puts.loc[0][i][0]['delta']) <= -0.5:
                put_straddle_bid_atm = puts.loc[0][i][0]['bid']
                straddle_strike_atm = puts.loc[0][i][0]['strikePrice']
                break
            i+=1
        i=0
        while True:
            if float(calls.loc[0][i][0]['strikePrice']) == straddle_strike_atm:
                call_straddle_bid_atm = calls.loc[0][i][0]['bid']
                break
            i+=1
        straddle_price_atm = call_straddle_bid_atm + put_straddle_bid_atm
        
        # get shorts
        i=0
        while True:
            if float(puts.loc[0][i][0]['delta']) <= -0.2 and puts.loc[0][i][0]['delta'] >= -0.4:
                put_strangle_bid = puts.loc[0][i][0]['bid']
                short_put_strike = puts.loc[0][i][0]['strikePrice']
                straddlestrike_put_strike = puts.loc[0][i][0]['strikePrice']
                # check the price of the ask
                if put_strangle_bid < 0.3:
                    i+=1
                    continue
                else:
                    break     
            i+=1
        
        i=1
        while True:
            if float(calls.loc[0][-i][0]['delta']) >= 0.2 and calls.loc[0][-i][0]['delta']<=0.4:
                call_strangle_bid = calls.loc[0][-i][0]['bid']
                short_call_strike=calls.loc[0][-i][0]['strikePrice']
                straddlestrike_call_strike = calls.loc[0][-i][0]['strikePrice']
                # check the price of the ask
                if call_strangle_bid < 0.3:
                    i+=1
                    continue
                else:
                    break
            i+=1
                
        #Check straddle prices 
        i=0
        while True:
            if float(puts.loc[0][i][0]['strikePrice']) == straddlestrike_put_strike:
                put_straddle_price = puts.loc[0][i][0]['bid'] + calls.loc[0][i][0]['bid']
                break
            i+=1
        i=0
        while True:
            if float(puts.loc[0][i][0]['strikePrice']) == straddlestrike_call_strike:
                call_straddle_price = puts.loc[0][i][0]['bid'] + calls.loc[0][i][0]['bid']
                break
            i+=1

        short_strangle_price = put_strangle_bid + call_strangle_bid
        
        if put_straddle_price*1.05 > straddle_price_atm and call_straddle_price*1.05 > straddle_price_atm:
            min_credit_percent = option_chain_skew(ticker)
            credit_percent = (short_strangle_price/straddle_price_atm)*100
            if credit_percent > min_credit_percent:
                print(ticker,': Trade Opportunity!')
                print(' Put Strike: ',short_put_strike)
                print(' Call Strike: ',short_call_strike)
                print(' Short strangle price:', short_strangle_price)
                print("")
            else:
                print(ticker,': Not enough credit %')
            
        else:
            print(ticker,': DONT TRADE')
            
        time.sleep(0.5)
    return 

In [79]:
for i in tqdm(temp_list):
    short_strangle_scanner(i)

--- AMZN ---
 straddle_delta_20_strike:  2980.0
 straddle_delta_20:         177.05
 straddle_delta_atm_strike: 3125.0
 straddle_delta_atm:        132.0
 skew_percent:              34.1 %
 min_credit_percent:        26

AMZN : Trade Opportunity!
 Put Strike:  2980.0
 Call Strike:  3265.0
 Short strangle price: 38.55

--- GM ---
 straddle_delta_20_strike:  37.5
 straddle_delta_20:         2.53
 straddle_delta_atm_strike: 40.0
 straddle_delta_atm:        2.07
 skew_percent:              22.2 %
 min_credit_percent:        28

GM : Trade Opportunity!
 Put Strike:  38.0
 Call Strike:  41.5
 Short strangle price: 0.75

--- PINS ---
 straddle_delta_20_strike:  55.5
 straddle_delta_20:         5.41
 straddle_delta_atm_strike: 60.0
 straddle_delta_atm:        4.36
 skew_percent:              24.1 %
 min_credit_percent:        28

PINS : Trade Opportunity!
 Put Strike:  55.5
 Call Strike:  65.0
 Short strangle price: 1.27

SNAP : Trade Opportunity!
 Put Strike:  38.5
 Call Strike:  44.5
 Short st